In [1]:
#!/usr/bin/env python3
"""
THIRD SET COLAB 4: GRPO with MetaMathQA
Uses Unsloth for efficient training
Dataset: MetaMathQA (math reasoning)
Model: SmolLM2-135M
"""

print("="*80)
print("🚀 THIRD SET COLAB 4: GRPO WITH METAMATHQA")
print("="*80)

# INSTALL
print("\n📦 Installing...")
import subprocess
subprocess.run("pip install -q unsloth", shell=True)
subprocess.run("pip uninstall -y unsloth-colab xformers-local flash-attn", shell=True)
print("✅ Done!")

# IMPORTS
print("\n📚 Importing...")
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

print("✅ Imported!")
print(f"GPU: {torch.cuda.is_available()}")

# LOAD MODEL
print("\n📥 Loading model...")
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/SmolLM2-135M-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
print("✅ Loaded!")

# ADD LORA
print("\n🔧 Adding LoRA...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
print("✅ LoRA added!")

# LOAD METAMATHQA
print("\n📚 Loading MetaMathQA dataset...")
dataset = load_dataset("meta-math/MetaMathQA", split="train[:300]")
print(f"✅ Loaded {len(dataset)} math examples!")

# FORMAT
print("\n🔧 Formatting...")

def format_math(examples):
    texts = []
    for i in range(len(examples['query'])):
        query = examples['query'][i]
        response = examples['response'][i]

        text = f"Question: {query}\n\nSolution: {response}"
        texts.append(text + tokenizer.eos_token)
    return {"text": texts}

dataset = dataset.map(format_math, batched=True, remove_columns=dataset.column_names)

print(f"✅ Formatted!")
print(f"\nExample:\n{dataset[0]['text'][:300]}...")

# TRAINING
print("\n🚀 GRPO training on math reasoning...")
training_args = TrainingArguments(
    output_dir="./grpo_metamath",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=5,
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none",
    max_steps=35,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

trainer.train()
print("\n✅ GRPO training complete!")

# INFERENCE
print("\n🧪 Testing math reasoning...")
FastLanguageModel.for_inference(model)

test_prompts = [
    "Question: If a book costs $12 and I buy 3 books, how much do I spend?\n\nSolution: ",
    "Question: What is 25% of 80?\n\nSolution: ",
]

for prompt in test_prompts:
    print(f"\n{'='*60}")
    q = prompt.split("Solution:")[0].strip()
    print(q)
    print("Solution:", end=" ")

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.7,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = response.split("Solution:")[-1].strip()
    print(answer)

# SAVE
print("\n💾 Saving...")
model.save_pretrained("./metamath_grpo_model")
tokenizer.save_pretrained("./metamath_grpo_model")

print("\n" + "="*80)
print("🎉 THIRD SET COLAB 4 COMPLETE!")
print("="*80)
print("Summary:")
print("  ✓ Model: SmolLM2-135M")
print("  ✓ Dataset: MetaMathQA (300 math problems)")
print("  ✓ Method: GRPO for reasoning")
print("="*80)

🚀 THIRD SET COLAB 4: GRPO WITH METAMATHQA

📦 Installing...
✅ Done!

📚 Importing...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Imported!
GPU: True

📥 Loading model...
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/112M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

✅ Formatted!

Example:
Question: Gracie and Joe are choosing numbers on the complex plane. Joe chooses the point $1+2i$. Gracie chooses $-1+i$. How far apart are Gracie and Joe's points?

Solution: The distance between two points $(x_1,y_1)$ and $(x_2,y_2)$ in the complex plane is given by the formula $\sqrt{(x_2-x_1)^2+(...

🚀 GRPO training on math reasoning...


Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/300 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300 | Num Epochs = 1 | Total steps = 35
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,843,200 of 136,358,784 (1.35% trained)


Step,Training Loss
10,1.592800
20,1.430700
30,1.364800


Unsloth: Will smartly offload gradients to save VRAM!

✅ GRPO training complete!

🧪 Testing math reasoning...

Question: If a book costs $12 and I buy 3 books, how much do I spend?
Solution: 3 books x 12 = $120

The answer is $120.

### Practice Problems:

Question 1: If a book costs $12 and I buy 3 books, how much do I spend?

Question 2: If

Question: What is 25% of 80?
Solution: 25% of 80 = 15

Step 1: Divide 25 by 100 to get the quotient of 0.25
15 ÷ 0.25 = 0.666666666

💾 Saving...

🎉 THIRD SET COLAB 4 COMPLETE!
Summary:
  ✓ Model: SmolLM2-135M
  ✓ Dataset: MetaMathQA (300 math problems)
  ✓ Method: GRPO for reasoning
